In [3]:
import os
import numpy as np
import gensim

In [4]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname) as f:
        for i, line in enumerate(f):
            tokens = line.split()
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(os.path.join('dataset', 'ogbg-molhiv.txt')))

SyntaxError: incomplete input (2740791177.py, line 13)

In [ ]:
len(train_corpus)

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(
    vector_size=1000,
    window=5
    )

In [ ]:
model.build_vocab(train_corpus)

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
X = [model.dv[node] for node in range(len(model.dv))]

In [ ]:
from ogb.graphproppred import GraphPropPredDataset

###

d_name = 'ogbg-molhiv'
dataset = GraphPropPredDataset(name=d_name)
split_idx = dataset.get_idx_split()

y = [data[1] for data in dataset]

In [ ]:
train_idx = split_idx['train']
test_idx = split_idx['test']

train_X = np.array([X[i] for i in range(len(X)) if i in train_idx])
test_X = np.array([X[i] for i in range(len(X)) if i in test_idx])

train_y = np.array([y[i] for i in range(len(y)) if i in train_idx]).ravel()
test_y = np.array([y[i] for i in range(len(y)) if i in test_idx]).ravel()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_X, train_y)

In [ ]:
train_y.shape

In [ ]:
y_pred = clf.predict(test_X)

In [ ]:
np.bincount(y_pred)

In [ ]:
np.bincount(test_y)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, clf.predict(test_X))